In [2]:
from pyspark.sql import SparkSession # SparkSession: 스파크 코드를 실행하기 위한 진입점
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [5]:
df = spark.read.format("csv")\
    .option("header","true")\
    .option("inferSchema","true")\
    .load("/Users/taewoong/Documents/coding/Spark_practice/data/retail-data/all/*.csv")\
    .coalesce(5)

### 윈도우 함수

위도우 함수를 집계에 사용할 수도 있다. 윈도우 함수는 데이터의 특정 'window'를 대상으로 고유의 집계 연산을 수행한다. 데이터의 '위도우'는 현재 데이터에 대한 참조를 사용해 정의 한다. 원도우 명세는 함수에 전달될 로우를 결정한다. 표준 group-by 함수와 유사해보일 수도 있으므로 이 둘의 차이점을 보겠다.

group-by함수를 사용하면 모든 로우 레크도가 단일 그룹으로만 이동한다. 윈도우 함수는 프레임에 입력되는 모든 로우에 대한 결과값을 계산한다. 프레임은 로우 그룹 기반의 테이블을 의미한다. 각 로우는 하나 이상의 프레임에 할당될 수 있다. 가장 흔하게 사용되는 방법 중 하나는 하루를 나타내는 값의 롤링 평균을 구하는 것이다. 이 작업을 수행하려면 개별 로우가 7개의 다른 프레임으로 구성되어야 한다.

스파크는 다음 세 가지 종류의 윈도우 함수를 지원한다.

* 랭크 함수
* 분석 함수
* 집계 한수


![](./IMG_BFA0D4F05227-1.jpeg)

In [7]:
from pyspark.sql.functions import col, to_date

In [8]:
dfWithDate = df.withColumn("date", to_date(col("InvoiceDate"), "MM/d/yyyy H:mm"))
dfWithDate.createOrReplaceTempView("dfWithDate")

윈도우 함수를 정의하기 위해 첫 번째 단계로 윈도우 명세를 만든다. 여기서 사용하는 partitionBy메서드는 지금까지 사용해온 파티셔닝 스키마의 개념과는 관련이 없으며 그룹을 어떻게 나눌지 결정하는 것과 유사한 개념이다. orderBy 메서드는 파티션의 정렬 방식을 정의한다. 그리고 프레임 명세(rowsBetween 구문)는 입력된 로우의 참조를 기반으로 프레임에 로우가 포함될 수 있는지 경정한다.

In [9]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc

In [10]:
windowSpec = Window\
  .partitionBy("CustomerId", "date")\
  .orderBy(desc("Quantity"))\
  .rowsBetween(Window.unboundedPreceding, Window.currentRow)

In [11]:
from pyspark.sql.functions import max

maxPurchaseQuantity = max(col("Quantity")).over(windowSpec)


In [12]:
from pyspark.sql.functions import dense_rank, rank

앞 예제는 컬럼이나 표현식을 반환하므로 DataFrame의 select 구문에서 사용할 수 있다. dense_rank 함수를 사용해 모든 고객에 대해 최대 구매 수량을 가진 날짜가 언제인지 알아본다. 동일한 값이 나오거나 중복 로우가 발생해 순위가 비어 있을 수 있으며로 rank함수 대산 dense_rank함수를 사용한다.

In [13]:
purchaseDenseRank = dense_rank().over(windowSpec)
purchaseRank = rank().over(windowSpec)

In [20]:
# 레거시 파서를 사용하는 대신 MM / dd / yyyy에서 MM-dd-yyyy
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
dfWithDate.where("CustomerId IS NOT NULL").orderBy("CustomerId")\
  .select(
    col("CustomerId"),
    col("date"),
    col("Quantity"),
    purchaseRank.alias("quantityRank"),
    purchaseDenseRank.alias("quantityDenseRank"),
    maxPurchaseQuantity.alias("maxPurchaseQuantity")).show(2)

+----------+----------+--------+------------+-----------------+-------------------+
|CustomerId|      date|Quantity|quantityRank|quantityDenseRank|maxPurchaseQuantity|
+----------+----------+--------+------------+-----------------+-------------------+
|     12346|2011-01-18|   74215|           1|                1|              74215|
|     12346|2011-01-18|  -74215|           2|                2|              74215|
+----------+----------+--------+------------+-----------------+-------------------+
only showing top 2 rows



### 그룹화 셋

그룹화 셋은 여러 집계를 결합하는 저수준 기증이다. 그룹화 셋을 이용하면 group-by 구문에서 원하는 형태로 집계를 생성할 수 있다.

In [17]:
dfNoNull = dfWithDate.drop()
dfNoNull.createOrReplaceTempView("dfNoNull")
dfNoNull.show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|      date|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|2010-12-01|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-12-01|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|2010-12-01|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-12-01|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-12-01|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|U

### 롤업

지금까지는 명시적 그룹화를 알아보았다. 다양한 컬럼을 그룹화 키로 설정하면 그룹화 키로 설정된 조합뿐만 아니라 데이터셋이세 볼 수 있는 실제 조합을 모두 살펴볼수 있다 롤업은 group-by 스타일의 다양한 연산을 수행할 수 있는 다차원 집계 기능이다

In [24]:
rolledUpDF = dfNoNull.rollup("Date", "Country").agg(sum(col("Quantity")))\
  .selectExpr("Date", "Country", "`sum(Quantity)` as total_quantity")\
  .orderBy("Date")
rolledUpDF.show()

+----------+--------------+--------------+
|      Date|       Country|total_quantity|
+----------+--------------+--------------+
|      null|          null|       5176450|
|2010-12-01|   Netherlands|            97|
|2010-12-01|     Australia|           107|
|2010-12-01|          null|         26814|
|2010-12-01|        France|           449|
|2010-12-01|          EIRE|           243|
|2010-12-01|       Germany|           117|
|2010-12-01|        Norway|          1852|
|2010-12-01|United Kingdom|         23949|
|2010-12-02|          EIRE|             4|
|2010-12-02|       Germany|           146|
|2010-12-02|          null|         21023|
|2010-12-02|United Kingdom|         20873|
|2010-12-03|        Poland|           140|
|2010-12-03|       Belgium|           528|
|2010-12-03|       Germany|           170|
|2010-12-03|         Spain|           400|
|2010-12-03|         Italy|           164|
|2010-12-03|      Portugal|            65|
|2010-12-03|          null|         14830|
+----------

### 큐브

큐브는 롤업을 고차원적으로 사용할 수 있게 해준다. 큐브는 요소들을 계층적으로 다루는 대신 모든 차원에 대해 동일한 작업을 수행한다.

In [19]:
from pyspark.sql.functions import sum

dfNoNull.cube("Date", "Country").agg(sum(col("Quantity")))\
  .select("Date", "Country", "sum(Quantity)").orderBy("Date").show()

+----+--------------------+-------------+
|Date|             Country|sum(Quantity)|
+----+--------------------+-------------+
|null|           Australia|        83653|
|null|               Japan|        25218|
|null|               Italy|         7999|
|null|                 USA|         1034|
|null|             Germany|       117448|
|null|         Unspecified|         3300|
|null|                null|      5176450|
|null|             Lebanon|          386|
|null|           Hong Kong|         4769|
|null|                 RSA|          352|
|null|           Singapore|         5234|
|null|     Channel Islands|         9479|
|null|               Spain|        26824|
|null|             Finland|        10666|
|null|  European Community|          497|
|null|             Denmark|         8188|
|null|              Cyprus|         6317|
|null|      Czech Republic|          592|
|null|United Arab Emirates|          982|
|null|              Norway|        19247|
+----+--------------------+-------

### 그룹화 메타 데이터

큐브와 롤업을 사용하다 보면 집계 수준에 따라 쉽게 필터링하기 위해 집계 수준을 조회하는 경우가 발생한다. 이때 grouping_id를 사용한다. grouping_id는 결과 데이터섹의 집계 수준을 명시하는 컬럼을 제공한다.

### 피벗

피벗을 사용해 로우를 컬럼으러 변환할 수 있다. 현재 데이터셋에는 Country 컬럼이 있다 피벗을 사용해 국가별로 집계 함수를 적용할 수 있으며 쿼리슷 사용해 쉽게 결과를 확인할 수 있다,

In [36]:
pivoted = dfWithDate.groupBy("date").pivot("Country").sum()

In [ ]:
pivoted.where("date > '2011-12-05'").select("date","'Norway_sum(Quantity)'").show()